# Optimizing realsense parameters for data acquisition

The Intel Realsense D415 comes supplied with many parameters that affect the quality of the depth acquisition. This can be a huge pain to tune by hand.

The goal of this notebook is to use python bindings to the realsense library and optimize parameters to improve the quality of our mouse recordings.

In [1]:
# first make sure the realsense python bindings are importable by python
# download libreasense sdk using the following instructions (For Linux):

# 1. open terminal
# 2. $ git clone https://github.com/IntelRealSense/librealsense
# 3. $ cd librealsense
# 4. $ mkdir build
# 5. $ cd build

# in order to get import pyrealsense2 to work you need to cmake with bindings that specify the full path of the python executable
# in order to find the full path, open python3 in the terminal:

# 6. $ python3  #this opens the python environment
# 7. >>>import sys
# 8. >>>print(sys.executable)
#       /usr/bin/python
# 9. >>>quit()

# if you are working in ipython notebook, you can do the above commands right in the notebook

In [2]:
import sys
print(sys.executable)

/usr/bin/python3


In [3]:
# now, when you use cmake (remember you're still in the build directory), copy the full path after -DBUILD_PYTHON_EXECUTABLE=[full path] as follows:

# 10. $ cmake ../ -DBUILD_PYTHON_BINDINGS=TRUE -DBUILD_PYTHON_EXECUTABLE=/home/rockwell/miniconda3/bin/python
# 11. $ make -j4
# 12. $ sudo make install #Optional if you want the library to be installed in your system

In [4]:
sys.path.append('/usr/local/lib')

In [5]:
import pyrealsense2 as rs

In [6]:
import numpy as np
import cv2
import json
import time

In [7]:
#values of some preset parameters. This is the list that will change in very loop of the SGD
values= [400, 0.461914, -0.553711, -0.553711, 0.0458984, 0.540039, 0.540039, 0.208008, -0.332031, -0.212891, -0.212891, 0.68457, 0.930664, 65536, 624, 53, "True", 33000, "True", 166, 0, 0 , 50, 64, 300, 0, 3, 64, 50, "True", 4600, 16, "False", 150, "on", 0, 400, 9, 9, 9, 9, 65536, 624, 901, 0, 0, 0, 0, 0, 0, 53, 800, 26, 24, 2047, 500, 1, 7, 1, 1, 3, 1, 1, 3, 0.0499022, 0.0499022, 0.0499022, 3, 1, 10, 10, 4, 1, 0.375, 72, 72, 72, 60, 105, 70, 342, 190, 130, 325, 0, 0, 1, 901] 

In [8]:
# KEY FUNCTION FOR SETTING THE D415
def writeD415Json(path,filename,values):
    filepathname= './' + path + filename + '.json'
    parameters={}
    parameters["aux-param-autoexposure-setpoint"]=str(values[0])
    parameters["aux-param-colorcorrection1"]=str(values[1])
    parameters["aux-param-colorcorrection10"]=str(values[2])
    parameters["aux-param-colorcorrection11"]=str(values[3])
    parameters["aux-param-colorcorrection12"]=str(values[4]) 
    parameters["aux-param-colorcorrection2"]=str(values[5])
    parameters["aux-param-colorcorrection3"]=str(values[6])
    parameters["aux-param-colorcorrection4"]=str(values[7]) 
    parameters["aux-param-colorcorrection5"]=str(values[8])
    parameters["aux-param-colorcorrection6"]=str(values[9])
    parameters["aux-param-colorcorrection7"]=str(values[10])
    parameters["aux-param-colorcorrection8"]=str(values[11])
    parameters["aux-param-colorcorrection9"]=str(values[12])
    parameters["aux-param-depthclampmax"]=str(values[13])
    parameters["aux-param-depthclampmin"]=str(values[14])
    parameters["aux-param-disparityshift"]=str(values[15])
    parameters["controls-autoexposure-auto"]=str(values[16])
    parameters["controls-autoexposure-manual"]=str(values[17])
    parameters["controls-color-autoexposure-auto"]=str(values[18])
    parameters["controls-color-autoexposure-manual"]=str(values[19])
    parameters["controls-color-backlight-compensation"]=str(values[20])
    parameters["controls-color-brightness"]=str(values[21])
    parameters["controls-color-contrast"]=str(values[22])
    parameters["controls-color-gain"]=str(values[23])
    parameters["controls-color-gamma"]=str(values[24])
    parameters["controls-color-hue"]=str(values[25])
    parameters["controls-color-power-line-frequency"]=str(values[26])
    parameters["controls-color-saturation"]=str(values[27])
    parameters["controls-color-sharpness"]=str(values[28])
    parameters["controls-color-white-balance-auto"]=str(values[29])
    parameters["controls-color-white-balance-manual"]=str(values[30])
    parameters["controls-depth-gain"]=str(values[31])
    parameters["controls-depth-white-balance-auto"]=str(values[32])
    parameters["controls-laserpower"]=str(values[33])
    parameters["controls-laserstate"]=str(values[34])
    parameters["ignoreSAD"]=str(values[35])
    parameters["param-autoexposure-setpoint"]=str(values[36])
    parameters["param-censusenablereg-udiameter"]=str(values[37])
    parameters["param-censusenablereg-vdiameter"]=str(values[38])
    parameters["param-censususize"]=str(values[39])
    parameters["param-censusvsize"]=str(values[40])
    parameters["param-depthclampmax"]=str(values[41]) 
    parameters["param-depthclampmin"]=str(values[42]) 
    parameters["param-depthunits"]=str(values[43])
    parameters["param-disableraucolor"]=str(values[44])
    parameters["param-disablesadcolor"]=str(values[45])
    parameters["param-disablesadnormalize"]=str(values[46])
    parameters["param-disablesloleftcolor"]=str(values[47])
    parameters["param-disableslorightcolor"]=str(values[48]) 
    parameters["param-disparitymode"]=str(values[49])
    parameters["param-disparityshift"]=str(values[50])
    parameters["param-lambdaad"]=str(values[51])
    parameters["param-lambdacensus"]=str(values[52])
    parameters["param-leftrightthreshold"]=str(values[53])
    parameters["param-maxscorethreshb"]=str(values[54])
    parameters["param-medianthreshold"]=str(values[55])
    parameters["param-minscorethresha"]=str(values[56])
    parameters["param-neighborthresh"]=str(values[57])
    parameters["param-raumine"]=str(values[58])
    parameters["param-rauminn"]=str(values[59])
    parameters["param-rauminnssum"]=str(values[60])
    parameters["param-raumins"]=str(values[61])
    parameters["param-rauminw"]=str(values[62])
    parameters["param-rauminwesum"]=str(values[63])
    parameters["param-regioncolorthresholdb"]=str(values[64])
    parameters["param-regioncolorthresholdg"]=str(values[65])
    parameters["param-regioncolorthresholdr"]=str(values[66]) 
    parameters["param-regionshrinku"]=str(values[67])
    parameters["param-regionshrinkv"]=str(values[68])
    parameters["param-robbinsmonrodecrement"]=str(values[69])
    parameters["param-robbinsmonroincrement"]=str(values[70])
    parameters["param-rsmdiffthreshold"]=str(values[71])
    parameters["param-rsmrauslodiffthreshold"]=str(values[72]) 
    parameters["param-rsmremovethreshold"]=str(values[73]) 
    parameters["param-scanlineedgetaub"]=str(values[74])
    parameters["param-scanlineedgetaug"]=str(values[75]) 
    parameters["param-scanlineedgetaur"]=str(values[76]) 
    parameters["param-scanlinep1"]=str(values[77]) 
    parameters["param-scanlinep1onediscon"]=str(values[78]) 
    parameters["param-scanlinep1twodiscon"]=str(values[79])
    parameters["param-scanlinep2"]=str(values[80])
    parameters["param-scanlinep2onediscon"]=str(values[81])
    parameters["param-scanlinep2twodiscon"]=str(values[82])
    parameters["param-secondpeakdelta"]=str(values[83])
    parameters["param-texturecountthresh"]=str(values[84])
    parameters["param-texturedifferencethresh"]=str(values[85])
    parameters["param-usersm"]=str(values[86]) 
    parameters["param-zunits"]=str(values[87])

    with open(filepathname, 'w') as fp:
        json.dump(parameters, fp)

filename='practice_writing_file'
path='./'
writeD415Json(path,filename,values)

In [9]:
values[41]

65536

In [10]:
# Enabling Advanced Mode

DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    
except Exception as e:
    print(e)
pass

Found device that supports advanced mode: Intel RealSense D415
Advanced mode is enabled


In [11]:
# Loading presaved parameters 
    
with open("practice_writing_file.json") as json_data: #Json file to be loaded
    J = json.load(json_data)
    J=str(J).replace("'", '\"') #IF YOU DON't DO THIS IT WILL NOT WORK with the librealsense sdk
    print(J)

{"param-rauminn": "1", "controls-laserstate": "on", "param-censususize": "9", "controls-color-autoexposure-manual": "166", "param-depthclampmin": "624", "aux-param-colorcorrection1": "0.461914", "controls-depth-white-balance-auto": "False", "controls-color-backlight-compensation": "0", "param-scanlinep1": "60", "controls-color-contrast": "50", "aux-param-colorcorrection11": "-0.553711", "controls-depth-gain": "16", "ignoreSAD": "0", "param-lambdaad": "800", "param-scanlineedgetaub": "72", "param-disparitymode": "0", "param-censusenablereg-udiameter": "9", "aux-param-autoexposure-setpoint": "400", "controls-color-power-line-frequency": "3", "param-autoexposure-setpoint": "400", "aux-param-colorcorrection3": "0.540039", "param-scanlinep2twodiscon": "130", "param-texturecountthresh": "0", "param-rsmrauslodiffthreshold": "1", "param-depthunits": "901", "controls-color-gain": "64", "param-raumins": "1", "param-rauminnssum": "3", "param-scanlinep2": "342", "param-rsmremovethreshold": "0.375"

In [12]:
print(J)

{"param-rauminn": "1", "controls-laserstate": "on", "param-censususize": "9", "controls-color-autoexposure-manual": "166", "param-depthclampmin": "624", "aux-param-colorcorrection1": "0.461914", "controls-depth-white-balance-auto": "False", "controls-color-backlight-compensation": "0", "param-scanlinep1": "60", "controls-color-contrast": "50", "aux-param-colorcorrection11": "-0.553711", "controls-depth-gain": "16", "ignoreSAD": "0", "param-lambdaad": "800", "param-scanlineedgetaub": "72", "param-disparitymode": "0", "param-censusenablereg-udiameter": "9", "aux-param-autoexposure-setpoint": "400", "controls-color-power-line-frequency": "3", "param-autoexposure-setpoint": "400", "aux-param-colorcorrection3": "0.540039", "param-scanlinep2twodiscon": "130", "param-texturecountthresh": "0", "param-rsmrauslodiffthreshold": "1", "param-depthunits": "901", "controls-color-gain": "64", "param-raumins": "1", "param-rauminnssum": "3", "param-scanlinep2": "342", "param-rsmremovethreshold": "0.375"

In [13]:
#Experimental Loop

frame_width=848
frame_height=480
fps=90
number_of_frames_to_record=180
file = open('pythondepthdata.bin', 'a+b') #open a file for apending binary data
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, fps)
#config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, fps)
advnc_mode.load_json(J)
# Start streaming
cfg = pipeline.start(config)
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
#depth_sensor.set_option(rs.option.visual_preset, 2)


iteration = 0;
preset = 0;
preset_name = '';
tick=0; #keeping track of the number of loops the experiment has gone through

start_of_loop = time.time()
advnc_mode.load_json(J) #LOADING ADVANCED MORE WITH OUR PRESET JSON J
with open('pythondepthdata.bin','ab') as binarydata: #While the file called "pythonedpthdata.bin" is open for appending binary, 
    try:
        while tick<number_of_frames_to_record: #aiming for 200 frames
            tick=tick+1 
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
        
            #iteration = iteration + 1
            #if iteration > 100:
               #preset = preset + 1
               #iteration = 0
               #range = depth_sensor.get_option_range(rs.option.visual_preset)
               #preset = preset % range.max
               #depth_sensor.set_option(rs.option.visual_preset, preset)
               #preset_name = depth_sensor.get_option_value_description(rs.option.visual_preset, preset)
        
            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            #color_image = np.asanyarray(color_frame.get_data())
            depth_image.tofile(binarydata) #save data as binarydata (specified by the with open as line)
        
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, None, 0.5, 0), cv2.COLORMAP_JET)

            # Stack both images horizontally
            #images = np.hstack((color_image, depth_colormap))
        
            #font = cv2.FONT_HERSHEY_SIMPLEX
            #cv2.putText(images, preset_name,(60,80), font, 4,(255,255,255),2,cv2.LINE_AA) 

            # Show images
            #cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            #cv2.imshow('RealSense', images)
            #cv2.waitKey(1)

    finally:

        # Stop streaming
        pipeline.stop()
        end_of_loop=time.time() #end timer
        print("Number of Experimental Loops = ", tick)
        print()
        print("Time Elapsed = ", end_of_loop - start_of_loop)
        serialized_string_2 = advnc_mode.serialize_json()
        as_json_object = json.loads(serialized_string_2)
        json_used_in_experiment = str(as_json_object).replace("'", '\"')
        print()
        print("Parameters Used = ", json_used_in_experiment)
       
      

Number of Experimental Loops =  180

Time Elapsed =  3.234884738922119

Parameters Used =  {"param-rauminn": "1", "controls-laserstate": "on", "param-censususize": "9", "controls-color-autoexposure-manual": "166", "param-depthclampmin": "624", "aux-param-colorcorrection1": "0.461914", "param-robbinsmonrodecrement": "10", "controls-color-backlight-compensation": "0", "param-scanlinep1": "60", "controls-color-contrast": "50", "aux-param-colorcorrection11": "-0.553711", "controls-depth-gain": "16", "ignoreSAD": "0", "param-lambdaad": "800", "param-disparitymode": "0", "param-censusenablereg-udiameter": "9", "aux-param-autoexposure-setpoint": "400", "controls-color-power-line-frequency": "3", "param-autoexposure-setpoint": "400", "aux-param-colorcorrection3": "0.540039", "param-scanlinep2twodiscon": "130", "param-texturecountthresh": "0", "param-depthunits": "901", "controls-color-gain": "64", "param-disablesadcolor": "0", "param-raumins": "1", "param-rauminnssum": "3", "param-rsmremovethr

In [14]:
!wc -c pythondepthdata.bin #count number of bytes in the file we just created. each pixel is a uint16 so divide bytes/framedimensions/2

146534400 pythondepthdata.bin


In [15]:
146534400/848/480/2 #should equal number of frames to record AND tick

180.0

In [16]:
#NOICE